In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
! ls

In [ ]:
! unzip /kaggle/input/dogs-vs-cats/test1.zip

In [ ]:
! unzip /kaggle/input/dogs-vs-cats/train.zip

In [ ]:
! unzip /kaggle/input/dogs-vs-cats/train.zip

In [ ]:
! ls

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import tensorflow as tf
import os
import glob
from tqdm import tqdm
from glob import glob
from shutil import copy, move

In [ ]:
train_path='train'
test_path='test1'

In [ ]:
train_set = os.listdir(train_path)
test_set = os.listdir(test_path)
print(len(train_set))

In [ ]:
dog_train_dir = './training/dog'
dog_val_dir = './validation/dog'
cat_train_dir = './training/cat'
cat_val_dir = './validation/cat'

file_dirs = [dog_train_dir, dog_val_dir, cat_train_dir, cat_val_dir]

for dir in file_dirs:
    os.makedirs(dir,exist_ok = True)

In [ ]:
all_dogs = glob('./train/dog*jpg')
all_cats = glob('./train/cat*jpg')

print(len(all_dogs))
print(len(all_cats))

In [ ]:
from sklearn.model_selection import train_test_split

dogs_train, dogs_test = train_test_split(all_dogs, test_size=0.1, shuffle=True,random_state = 100)
cats_train, cats_test = train_test_split(all_cats, test_size=0.1, shuffle=True,random_state = 100)

In [ ]:
for name in tqdm(dogs_train):
  copy(name , os.path.join(dog_train_dir , name.split('/')[-1]))
for name in tqdm(cats_train):
  copy(name , os.path.join(cat_train_dir , name.split('/')[-1]))
  
for name in tqdm(dogs_test):
  copy(name , os.path.join(dog_val_dir , name.split('/')[-1]))
for name in tqdm(cats_test):
  copy(name , os.path.join(cat_val_dir , name.split('/')[-1]))

In [ ]:
print(len(os.listdir(dog_train_dir)))
print(len(os.listdir(cat_train_dir)))
print(len(os.listdir(dog_val_dir)))
print(len(os.listdir(cat_val_dir)))

In [ ]:
def plot_imgs(item_dir, top=10):
    all_item_dirs = os.listdir(item_dir)
    item_files = [os.path.join(item_dir, file) for file in all_item_dirs][100:106]
  
    plt.figure(figsize=(10, 10))
  
    for idx, img_path in enumerate(item_files):
        plt.subplot(1, 6, idx+1)        
        img = plt.imread(img_path)
        plt.tight_layout() 
        plt.axis('off')
        plt.imshow(img, cmap='gray')

In [ ]:
img_height = 224
img_width = 224

training_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
                                                                    rescale=1/255.0,
                                                                    validation_split=0.15)
train_path="./training"
train_generator = training_data_gen.flow_from_directory(train_path,
                                                        target_size=(img_height, img_width),
                                                        batch_size=64,
                                                        shuffle=True,
                                                        subset='training') # set as training data

validation_generator = training_data_gen.flow_from_directory(train_path, # same directory as training data
                                                             target_size=(img_height, img_width),
                                                             batch_size=64,
                                                             shuffle=True,
                                                             subset='validation') # set as validation data
 
test_path="./validation"
testing_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0)

test_generator = testing_data_gen.flow_from_directory(test_path,
                                                      target_size=(img_height, img_width),
                                                      batch_size=60,
                                                      seed=0,
                                                      shuffle=False,
                                                      ) 

In [ ]:
img,label=next(train_generator)
print(label)
print(type(label))
#----------------------------------
# li=list(label)
# print(type(li))
# print(li)
labels=['cat','dog']
image = img[1]
label2 = int(label[1])

print(labels[label2])
plt.imshow(image)
plt.show()

In [ ]:
# to show images form img array
import cv2
%matplotlib inline
def plotImages(images_arr):
    fig, axes = plt.subplots(2, 4, figsize=(10,15))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        #ax.axis('off')
    plt.tight_layout()
    plt.show()
#----------------------------------------------------------------------------------------------------------------------------
# to show images by bath form directory  
fig = plt.figure(figsize=(15,10))
columns = 6
rows = 1
fielName = train_path

for i in range(columns*rows ):
      # each loop takes a different name each file in the directory in order so first time takes mountain, street, buildings,....
      folderName = os.path.join((fielName), os.listdir(fielName)[i])
      print(i)
      print(folderName)
      img = cv2.imread(folderName+'/'+(os.listdir(folderName)[i]))
      print(folderName+'/'+(os.listdir(folderName)[i]))
      fig.add_subplot(rows, columns, i+1)
      plt.imshow(img)
      plt.axis('off')
      #to print each image with the folder name
      plt.title(os.path.basename(folderName))

plt.show()

In [ ]:
plotImages(img)

In [ ]:
model=tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),strides=1,activation='relu',padding='same',input_shape=(224,224,3)))
model.add(tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding="same"))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=2))

model.add(tf.keras.layers.Conv2D(filters=128,kernel_size=(3,3),activation='relu',padding='same'))
model.add(tf.keras.layers.Conv2D(filters=128,kernel_size=(3,3),activation='relu',padding='same'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=2))

model.add(tf.keras.layers.Conv2D(filters=256,kernel_size=(3,3),activation='relu',padding='same'))
model.add(tf.keras.layers.Conv2D(filters=256,kernel_size=(3,3),activation='relu',padding='same'))
model.add(tf.keras.layers.Conv2D(filters=256,kernel_size=(3,3),activation='relu',padding='same'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=2))

model.add(tf.keras.layers.Conv2D(filters=512,kernel_size=(3,3),activation='relu',padding='same'))
model.add(tf.keras.layers.Conv2D(filters=512,kernel_size=(3,3),activation='relu',padding='same'))
model.add(tf.keras.layers.Conv2D(filters=512,kernel_size=(3,3),activation='relu',padding='same'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=2))

model.add(tf.keras.layers.Conv2D(filters=512,kernel_size=(3,3),activation='relu',padding='same'))
model.add(tf.keras.layers.Conv2D(filters=512,kernel_size=(3,3),activation='relu',padding='same'))
model.add(tf.keras.layers.Conv2D(filters=512,kernel_size=(3,3),activation='relu',padding='same'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=2))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(units=4096,activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(units=4096,activation="relu"))
model.add(tf.keras.layers.Dropout(0.5))
#model.add(tf.keras.layers.Dense(units=1000,activation="relu")))
model.add(tf.keras.layers.Dense(units=2,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
# from tensorflow import keras
# from tensorflow.keras import optimizers
# from tensorflow.keras.optimizers import schedules
# from keras.optimizers import SGD
# opt = SGD(lr=0.01)
# #model.compile(loss = "categorical_crossentropy", optimizer = opt)
model.compile(optimizer='adam',loss= 'categorical_crossentropy',metrics=['accuracy'])
# tf.keras.losses.binary_crossentropy

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])
history=model.fit(train_generator,validation_data=validation_generator, epochs=20,verbose=1)
#history=model.fit(train_generator,validation_data=validation_generator, epochs=20,verbose=1,callbacks=[checkpoint,early])
#hist = model.fit(train_generator, validation_data=validation_generator, epochs=100,callbacks=[checkpoint,early])

In [ ]:
from keras import backend as K
K.clear_session()